In [363]:
X = pd.DataFrame(
    {'fea1':[0,0,0,2,2,2,1,1,1],
     'fea2':['a',np.nan,'b','b','b','b',np.nan,'c','c'],
     'fea3':['Магазин','Магазин1','Магази','Что','ЧТОт','ЧТК','ЧАШК','ЧАШКК','чаш']})
Y = pd.DataFrame({'target':[1,1,1,1,0,1,0,0,0]})

In [364]:
X

,fea1,fea2,fea3
0,0,a,Магазин
1,0,NaN,Магазин1
2,0,b,Магази
3,2,b,Что
4,2,b,ЧТОт
5,2,b,ЧТК
6,1,NaN,ЧАШК
7,1,c,ЧАШКК
8,1,c,чаш


In [374]:
# from model.сonveyor import *
from sklearn.preprocessing import OrdinalEncoder
from typing import List
import pandas as pd
import numpy as np
####################################################################################################
class Column_scaler:
    """
    Pipeline of transforms with a final estimator.

    Sequentially apply a list of transforms and a final estimator.
    Intermediate steps of the pipeline must be 'transforms', that is, they
    must implement fit and transform methods.
    The final estimator only needs to implement fit.
    The transformers in the pipeline can be cached using ``memory`` argument.

    The purpose of the pipeline is to assemble several steps that can be
    cross-validated together while setting different parameters.
    For this, it enables setting parameters of the various steps using their
    names and the parameter name separated by a '__', as in the example below.
    A step's estimator may be replaced entirely by setting the parameter
    with its name to another estimator, or a transformer removed by setting
    it to 'passthrough' or ``None``.

    Read more in the :ref:`User Guide <pipeline>`.

    .. versionadded:: 0.5

    Parameters
    ----------
    steps : list
        List of (name, transform) tuples (implementing fit/transform) that are
        chained, in the order in which they are chained, with the last object
        an estimator.

    memory : str or object with the joblib.Memory interface, default=None
        Used to cache the fitted transformers of the pipeline. By default,
        no caching is performed. If a string is given, it is the path to
        the caching directory. Enabling caching triggers a clone of
        the transformers before fitting. Therefore, the transformer
        instance given to the pipeline cannot be inspected
        directly. Use the attribute ``named_steps`` or ``steps`` to
        inspect estimators within the pipeline. Caching the
        transformers is advantageous when fitting is time consuming.

    verbose : bool, default=False
        If True, the time elapsed while fitting each step will be printed as it
        is completed.

    Attributes
    ----------
    named_steps : :class:`~sklearn.utils.Bunch`
        Dictionary-like object, with the following attributes.
        Read-only attribute to access any step parameter by user given name.
        Keys are step names and values are steps parameters.

    See Also
    --------
    make_pipeline : Convenience function for simplified pipeline construction.

    Examples
    --------
    >>> from sklearn.svm import SVC
    >>> from sklearn.preprocessing import StandardScaler
    >>> from sklearn.datasets import make_classification
    >>> from sklearn.model_selection import train_test_split
    >>> from sklearn.pipeline import Pipeline
    >>> X, y = make_classification(random_state=0)
    >>> X_train, X_test, y_train, y_test = train_test_split(X, y,
    ...                                                     random_state=0)
    >>> pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
    >>> # The pipeline can be used as any other estimator
    >>> # and avoids leaking the test set into the train set
    >>> pipe.fit(X_train, y_train)
    Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])
    >>> pipe.score(X_test, y_test)
    0.88
    """
    ordinal_scaler = {}
    means_scaler = {}

    def __init__(self, columns_to_ordinal, columns_to_int):
        self.columns_to_ordinal = columns_to_ordinal
        self.columns_to_int = columns_to_int

    ## Выборка слов из датасета, и подача их на вход 
    def fit(self, X:pd.DataFrame, y:pd.Series):
        """Fit the model and transform with the final estimator

        Fits all the transforms one after the other and transforms the
        data, then uses fit_transform on transformed data with the final
        estimator.

        Parameters
        ----------
        X : iterable
            Training data. Must fulfill input requirements of first step of the
            pipeline.

        y : iterable, default=None
            Training targets. Must fulfill label requirements for all steps of
            the pipeline.

        **fit_params : dict of string -> object
            Parameters passed to the ``fit`` method of each step, where
            each parameter name is prefixed such that parameter ``p`` for step
            ``s`` has key ``s__p``.

        Returns
        -------
        Xt : array-like of shape  (n_samples, n_transformed_features)
            Transformed samples
        """
        for column in self.columns_to_ordinal:
            self.ordinal_scaler[column] = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
            self.ordinal_scaler[column].fit(X[column].to_frame().fillna('NAN').astype(str))
            ordinal_code = self.to_ordinal_scaler(X[column])
            self.means_scaler[column] = np.mean(ordinal_code[~np.isnan(ordinal_code)])
            
        for column in self.columns_to_int:
            self.means_scaler[column] = float(X[column].loc[~X[column].isnull()].mean())


        if not os.path.exists('model_new_property'):
            os.makedirs('model_new_property')
        with open('model_new_property/means_scaler', 'wb') as file:
            pickle.dump(self.means_scaler,file)
        with open('model_new_property/ordinal_scaler', 'wb') as file:
            pickle.dump(self.ordinal_scaler,file)
        return self

    def transform(self, X:pd.DataFrame, y = None)->pd.Series:
        with open('model_new_property/means_scaler', 'rb') as file:
            self.means_scaler = pickle.load(file)
        with open('model_new_property/ordinal_scaler', 'rb') as file:
            self.ordinal_scaler = pickle.load(file)

        for column in self.columns_to_ordinal:
            X[column] = self.to_ordinal_scaler(X[column])
            X[column].loc[X[column].isnull()] = self.means_scaler[column]
            
        for column in self.columns_to_int:
            X[column] = self.to_int_with_mean(X[column])
            X[column].loc[X[column].isnull()] = self.means_scaler[column]

        for column in X:
            X[column].loc[X[column].isnull()] = 0
        X['purpose_other'] =  X['purpose_other'].astype(str)
        return X
        

    def to_ordinal_scaler(self, series: pd.Series) -> pd.Series:
        frame = series.to_frame().fillna('NAN')
        frame = series.astype(str)
        return self.ordinal_scaler[series.name].transform(frame.values.reshape(-1,1))

    def to_int_with_mean(self, series: pd.Series) -> pd.Series:
        try:
            series = series.fillna(self.means_scaler[series.name])
            series = series.astype(int)
        except Exception as e:
            series = series.fillna(0)
            series = series.astype(int)
        return series
############################################################################################################

class Conveyor:
    def __init__(self, *blocks, **params):
        print("__init__")
        self.blocks = list(blocks)
        pass

    def train(self, X:pd.DataFrame, Y:pd.DataFrame or pd.Series):
        X_, Y_  = ( X.copy(), Y.copy() )
        for block in range(len(self.blocks)):
            self.blocks[block].fit(X_, Y_)
            self.blocks[block].transform(X_)

    def predict(self, X:pd.DataFrame, Y:pd.DataFrame or pd.Series = None):
        X_ =  X.copy() 
        for block in range(len(self.blocks)):
            self.blocks[block].transform(X_)
        print(X_)
    def export(self):
        pass

def most_frequency(x:List[List[int]]):
    x_ = [i[0] for i in x]
    return np.argmax(np.bincount(x_))

class CategoricalEncoder():
    encoder = {}

    def __init__(self, columns:pd.DataFrame, strategy:str='mean', fill_value:float or str = 0): # strategy in mean, median, most_frequency, const, iterative inputer?
        self.columns = columns
        self.fill_value = {'mean':np.mean, 'median':np.median, 'most_freq':most_frequency, 'const':(lambda x:fill_value)}
        self.fill_value = self.fill_value[strategy]

    def fit(self, X:pd.DataFrame, Y:pd.DataFrame or pd.Series):
        for column in self.columns:
            self.encoder[column] = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
            X_fit = pd.DataFrame(X[column].loc[~X[column].isnull()])
            self.encoder[column].fit(X_fit)
            X_transform = self.encoder[column].transform(pd.DataFrame(X_fit))
            self.encoder[column].unknown_value = self.fill_value(X_transform)
        return self

    def transform(self, X:pd.DataFrame):
        for column in self.columns:
            X[column] = self.encoder[column].transform(pd.DataFrame(X[column].fillna('NAN')))
        pass

class Imputer():
    encoder = {}

    def __init__(self, columns:pd.DataFrame, strategy:str='mean', fill_value:float or str = 0): # strategy in mean, median, most_frequency, const, iterative inputer?
        self.columns = columns
        self.fill_value = {'mean':np.mean, 'median':np.median, 'most_freq':most_frequency, 'const':(lambda x:fill_value)}
        self.fill_value = self.fill_value[strategy]

    def fit(self, X:pd.DataFrame, Y:pd.DataFrame or pd.Series):
        for column in self.columns:
            self.encoder[column] = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
            X_fit = pd.DataFrame(X[column].loc[~X[column].isnull()])
            self.encoder[column].fit(X_fit)
            X_transform = self.encoder[column].transform(pd.DataFrame(X_fit))
            self.encoder[column].unknown_value = self.fill_value(X_transform)
        return self

    def transform(self, X:pd.DataFrame):
        for column in self.columns:
            X[column] = self.encoder[column].transform(pd.DataFrame(X[column].fillna('NAN')))
        pass

In [375]:
model = Conveyor(CategoricalEncoder(columns=['fea2','fea3']))

__init__


In [380]:
model.train(X, Y)

In [390]:
X_test = pd.DataFrame(
    {'fea1':[0,0,1,2,5],
     'fea2':['a',np.nan,'b',np.nan,'c'],
     'fea3':['1','1','1','1','1']})
Y_test = pd.DataFrame({'target':[1,1,1,1,0]})

In [391]:
model.predict(X_test)

   fea1      fea2  fea3
0     0  0.000000   4.0
1     0  1.142857   4.0
2     1  1.000000   4.0
3     2  1.142857   4.0
4     5  2.000000   4.0


In [393]:
import pickle
with open('model_', 'wb') as save_file:
    pickle.dump(model, save_file)

In [394]:
with open('model_', 'rb') as load_file:
    model_test = pickle.load(load_file)

In [395]:
model_test

In [396]:
model_test.predict(X_test)

   fea1      fea2  fea3
0     0  0.000000   4.0
1     0  1.142857   4.0
2     1  1.000000   4.0
3     2  1.142857   4.0
4     5  2.000000   4.0
